# Tree Methods Comparison

In this lecture we will code along with some data and test out 3 different tree methods:

* A single decision tree
* A random forest
* A gradient boosted tree classifier
    
We will be using a college dataset to try to classify colleges as Private or Public

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('treeCodes').getOrCreate()

In [2]:
# Load training data
data = spark.read.csv('College.csv',inferSchema=True,header=True)

In [3]:
data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [4]:
data.head()

Row(School='Abilene Christian University', Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60)

### Spark Formatting of Data

In [5]:
# It needs to be in the form of two columns
# ("label","features")

# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [6]:
data.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [7]:
assembler = VectorAssembler(
  inputCols=['Apps',
             'Accept',
             'Enroll',
             'Top10perc',
             'Top25perc',
             'F_Undergrad',
             'P_Undergrad',
             'Outstate',
             'Room_Board',
             'Books',
             'Personal',
             'PhD',
             'Terminal',
             'S_F_Ratio',
             'perc_alumni',
             'Expend',
             'Grad_Rate'],
              outputCol="features")

In [8]:
output = assembler.transform(data)

Deal with Private column being "yes" or "no"

In [9]:
from pyspark.ml.feature import StringIndexer

In [10]:
indexer = StringIndexer(inputCol="Private", outputCol="PrivateIndex")
output_fixed = indexer.fit(output).transform(output)

In [11]:
final_data = output_fixed.select("features",'PrivateIndex')

In [13]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

### The Classifiers

In [14]:
from pyspark.ml.classification import DecisionTreeClassifier,GBTClassifier,RandomForestClassifier
from pyspark.ml import Pipeline

Create all three models:

In [15]:
# Use mostly defaults to make this comparison "fair"

dtc = DecisionTreeClassifier(labelCol='PrivateIndex',featuresCol='features')
rfc = RandomForestClassifier(labelCol='PrivateIndex',featuresCol='features')
gbt = GBTClassifier(labelCol='PrivateIndex',featuresCol='features')

Train all three models:

In [16]:
# Train the models (its three models, so it might take some time)
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

## Model Comparison

Let's compare each of these models!

In [17]:
dtc_predictions = dtc_model.transform(test_data)
rfc_predictions = rfc_model.transform(test_data)
gbt_predictions = gbt_model.transform(test_data)

**Evaluation Metrics:**

In [18]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [21]:
# For Accuracy
acc_evac = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='PrivateIndex',
                                             metricName='accuracy')

# For precision
prec_evac = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='PrivateIndex',
                                             metricName='weightedPrecision')

# For recall
rec_evac = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='PrivateIndex',
                                             metricName='weightedRecall')

# For f1 score
f1_evac = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='PrivateIndex',
                                             metricName='f1')

In [22]:
# DTC Metrics
acc_res = acc_evac.evaluate(dtc_predictions)

pres_res = prec_evac.evaluate(dtc_predictions)

recall_res = rec_evac.evaluate(dtc_predictions)

f1_res = f1_evac.evaluate(dtc_predictions)

In [24]:
print("DTC Test Accuracy Score = {:.2f}%".format(acc_res * 100))
print("DTC Test Error = {:.2f}%".format(1.0 - acc_res))
print()

print("DTC Test Precision Score = {:.2f}%".format(pres_res * 100))
print("DTC Test Error = {:.2f}%".format(1.0 - pres_res))
print()

print("DTC Test Recall Score = {:.2f}%".format(recall_res * 100))
print("DTC Test Error = {:.2f}%".format(1.0 - recall_res))
print()

print("DTC Test F1 Score = {:.2f}%".format(f1_res * 100))
print("DTC Test Error = {:.2f}%".format(1.0 - f1_res))

DTC Test Accuracy Score = 92.27%
DTC Test Error = 0.08%

DTC Test Precision Score = 92.24%
DTC Test Error = 0.08%

DTC Test Recall Score = 92.27%
DTC Test Error = 0.08%

DTC Test F1 Score = 92.12%
DTC Test Error = 0.08%


In [25]:
# RFC Metrics
acc_res = acc_evac.evaluate(rfc_predictions)

pres_res = prec_evac.evaluate(rfc_predictions)

recall_res = rec_evac.evaluate(rfc_predictions)

f1_res = f1_evac.evaluate(rfc_predictions)

In [26]:
print("RFC Test Accuracy Score = {:.2f}%".format(acc_res * 100))
print("RFC Test Error = {:.2f}%".format(1.0 - acc_res))
print()

print("RFC Test Precision Score = {:.2f}%".format(pres_res * 100))
print("RFC Test Error = {:.2f}%".format(1.0 - pres_res))
print()

print("RFC Test Recall Score = {:.2f}%".format(recall_res * 100))
print("RFC Test Error = {:.2f}%".format(1.0 - recall_res))
print()

print("RFC Test F1 Score = {:.2f}%".format(f1_res * 100))
print("RFC Test Error = {:.2f}%".format(1.0 - f1_res))

RFC Test Accuracy Score = 95.28%
RFC Test Error = 0.05%

RFC Test Precision Score = 95.34%
RFC Test Error = 0.05%

RFC Test Recall Score = 95.28%
RFC Test Error = 0.05%

RFC Test F1 Score = 95.20%
RFC Test Error = 0.05%


In [27]:
# GBT Metrics
acc_res = acc_evac.evaluate(gbt_predictions)

pres_res = prec_evac.evaluate(gbt_predictions)

recall_res = rec_evac.evaluate(gbt_predictions)

f1_res = f1_evac.evaluate(gbt_predictions)

In [28]:
print("GBT Test Accuracy Score = {:.2f}%".format(acc_res * 100))
print("GBT Test Error = {:.2f}%".format(1.0 - acc_res))
print()

print("GBT Test Precision Score = {:.2f}%".format(pres_res * 100))
print("GBT Test Error = {:.2f}%".format(1.0 - pres_res))
print()

print("GBT Test Recall Score = {:.2f}%".format(recall_res * 100))
print("GBT Test Error = {:.2f}%".format(1.0 - recall_res))
print()

print("GBT Test F1 Score = {:.2f}%".format(f1_res * 100))
print("GBT Test Error = {:.2f}%".format(1.0 - f1_res))

GBT Test Accuracy Score = 93.56%
GBT Test Error = 0.06%

GBT Test Precision Score = 93.51%
GBT Test Error = 0.06%

GBT Test Recall Score = 93.56%
GBT Test Error = 0.06%

GBT Test F1 Score = 93.52%
GBT Test Error = 0.06%


Interesting! Optional Assignment - play around with the parameters of each of these models, can you squeeze some more accuracy out of them? Or is the data the limiting factor?